In [1]:
!pip install -qU langchain-google-vertexai
!pip install -qU "langchain-community>=0.2.11" tavily-python

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [35]:
import getpass
import os
from dotenv import load_dotenv
load_dotenv()

True

TRAVILY PORTION

In [36]:
import getpass
import os
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

In [37]:
from langchain_community.tools import TavilySearchResults

tool = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True,
    # include_domains=[...],
    # exclude_domains=[...],
    # name="...",            # overwrite default tool name
    # description="...",     # overwrite default tool description
    # args_schema=...,       # overwrite default args_schema: BaseModel
)

In [38]:
product = "parle g biscuit" 
PROMPT = f"Give me the ingredients that are present in {product}"
tavily_reply = tool.invoke({"query": PROMPT})
first_reply = tavily_reply[0]
ingredients = first_reply.get('content')



In [40]:
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] =  " "
PROJECT_ID = " "  # @param {type:"string"}
REGION = " "  # @param {type: "string"}




In [43]:
import vertexai
vertexai.init(project=PROJECT_ID, location=REGION)

In [44]:
from IPython.display import Markdown, display
from vertexai.generative_models import (
    GenerationResponse,
    GenerativeModel,
    Tool,
    grounding,
)
from vertexai.preview.generative_models import grounding as preview_grounding

In [45]:
model = GenerativeModel("gemini-1.5-flash-001")

In [47]:
prompt_2 = f"The prodcut is {product} and the ingredients are {ingredients}, now do the following.Category 1: Can you give me a brief of taste on what the {product} is and  add that full info in(*** ***). Category 2: Also only list the nutritional (benefits/harms) if there are any. Category:3(only mention relevant category): look if the product is suitable for a vegan or keto or jain or all diets and flag only the nsame of that diet of any or multiple suitable in  (*** ***). Category 4(single word answer):If the {product} is organic or supports sustainablity or small businesses add that in (*** ***) . Category 5: ALso if there is any recent news regarding {product} flag it in less than 3 words in (*** ***). Give me the ingredietns of this {product} from the internet (*** ***)"
response = model.generate_content(prompt_2, tools=[tool])
print(response.candidates[0].content.parts[0].text)






     

TypeError: Unexpected tool type: include_answer=True include_raw_content=True include_images=True api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********')).

In [ ]:
tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())
response = model.generate_content(PROMPT, tools=[tool])
response_string = str(response)

In [32]:
product = "parle g biscuit" 
PROMPT = f"Give me the ingredients that are present in {product}"

In [33]:
tavily_reply = tool.invoke({"query": PROMPT})
tavily_reply[0].


{'url': 'https://statcarewalkin.com/info/parle-g-biscuits-nutritional-value.html',
 'content': "They also contain invert syrup, leavening agents, salt, wheat bran, emulsifiers, improvers, and yeast. These ingredients give Parle G biscuits their distinctive taste and texture. Calories. One Parle G biscuit contains approximately 33 calories. This makes it a relatively low-calorie snack option. However, it's important to note that the ..."}

In [5]:
import getpass
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = " "


PROJECT_ID = " "  # @param {type:"string"}
REGION = ""  # @param {type: "string"}
import vertexai
vertexai.init(project=PROJECT_ID, location=REGION)



In [6]:
from langchain_google_vertexai import ChatVertexAI

llm = ChatVertexAI(model="gemini-1.5-flash")

In [ ]:
tool_resulttool.invoke({"query": "What happened at the last wimbledon"})

In [19]:
import datetime

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

today = datetime.datetime.today().strftime("%D")
prompt = ChatPromptTemplate(
    [
        ("system", f"You are a thorough nutritionist who can anlyse the imgredients of food products and give the complete info as asked next"),
        ("human", "Category 1: Can you give me a brief of taste on what the {product} is and  add that full info in(*** ***). Category 2: Also only list the nutritional (benefits/harms) if there are any. Category:3(only mention relevant category): look if the product is suitable for a vegan or keto or jain or all diets and flag only the nsame of that diet of any or multiple suitable in  (*** ***). Category 4(single word answer):If the {product} is organic or supports sustainablity or small businesses add that in (*** ***) . Category 5: ALso if there is any recent news regarding {product} flag it in less than 3 words in (*** ***). Give me the ingredietns of this {product} from the internet (*** ***)")
        
    ]
)

# specifying tool_choice will force the model to call this tool.
llm_with_tools = llm.bind_tools([tool])

llm_chain = prompt | llm_with_tools


@chain
def tool_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    tool_msgs = tool.batch(ai_msg.tool_calls, config=config)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)



llm_chain.invoke(
    {
        
        "image_data": image_data,
        "age":"34",
        "weight": "168kg",
        "gender": "male",
        "height":"5 ft",
        "diet_type": "jain",
        "health_goal": "loose 50 kgs",
        "allergen": "peanuts",
        "product_info":"Pringles Potato Chips Desi Masala Tadka Flavor Pack of 4, Crispy Snack for Game Nights Chips (4 x 102 g)",
        "product_info_str": response_grounding
        

    }
)

AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "Chings Secret Veg Hakka Noodles, 6x150 g Multipack"}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_LOW'}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_LOW'}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}], 'usage_metadata': {'prompt_token_count': 302, 'candidates_token_count': 26, 'total_token_count': 328, 'cached_content_token_count': 0}, 'finish_reason': 'STOP'}, id='run-f2797bfd-3887-4395-8426-d24b2b85e188-0', tool_call